# Processing Montserrat CSV catalog with AAA
---

**Last update:** 12/2021 - Glenn THOMPSON  
**Contact:** thompsong@usf.edu   
---

In [1]:
################################
# General setup                #
################################

import os
import pandas as pd
import pickle
import numpy as np


# Change if you want your screen to keep quiet
# 0 = quiet
# 1 = in between
# 2 = detailed information
verbatim = 2

### PREPARE THE CATALOG DataFrame ###
SEISAN_DATA = os.path.join( os.getenv('HOME'),'DATA','MVO') # e.g. /home/user/seismo
pandaSeisDir = os.path.join(SEISAN_DATA, 'miniseed_c') # e.g. /home/user/seismo/pandaSeis
SEISAN_DB = 'MVOE_' # e.g. the seisan database name (e.g. MVOE_)
PROJECTDIR = os.path.join(os.getenv('HOME'),'src', 'kitchensinkGT', 'PROJECTS', 'MontserratML') # this dir
#csvfile_external = os.path.join(PROJECTDIR, 'MVO_labelled_events.csv')
csvfile_external = os.path.join(SEISAN_DATA, 'MachineLearning', SEISAN_DB, 'runAAA', 'MVOE_11_labelled_events.csv')
#csvfile_internal = './catalog/MVO_labelled_events_filtered.csv'
csvfile_internal = 'catalog/30_MVO_labelled_events_filtered.csv' # has to match that in AAA-master/config/general/newsettings_10.json
csvfile_internal = './AAA-master/MONTSERRAT/' + csvfile_internal
output_path_cat = csvfile_internal.replace('.csv', '.pd')
alltraces_file = '30_alltraceDFs.csv'

################################
# Machine learning and testing #
################################
import sys
sys.path.insert(0, './AAA-master/automatic_processing')
#import tools
from config import Config
from analyzer import Analyzer

# Change if you want your screen to keep quiet
# 0 = quiet
# 1 = in between
# 2 = detailed information
verbatim = 2

# Init project with configuration file
config = Config('./AAA-master/config/general/newsettings_10.json', verbatim=verbatim)
config.readAndCheck()  

##########################
# Variables to loop over #
##########################
alltraces = pd.read_csv(alltraces_file)
traceIDs = ['MV.MBWH..SHZ', 'MV.MBLG..SHZ', 'MV.MBRY..SHZ']
minWeights = range(4)
classes_to_include = [ ['l', 't'], ['e', 'r'], ['h', 'l', 't'], ['h', 'l', 't', 'r'], ['e', 'h', 'l', 't', 'r'] ]

traceIDs = ['MV.MBWH..SHZ']
minWeights = range(1)
classes_to_include = [ ['e', 'h', 'l', 't', 'r'] ]


#############
# Functions #
#############

def cat_filter_traceID(cat, alltraces, traceID):
    # subset catalog based on traceID
    matchingEvents = alltraces[alltraces['id']==traceID]
    for i, row in cat.iterrows():
        matching_indices = matchingEvents.index[matchingEvents['filetime']==row['filetime']].tolist()
        if len(matching_indices)==1:
            pass
        else:
            cat.drop(i, inplace=True)
    N = len(cat.index)
    #print('%d events after matching against traceID' % N)
    return N

def cat_filter_classes(cat, remove_classes):
    """
    for rmclass in remove_classes:
        print('Removing %s' % rmclass)
        cat = cat[cat['class']!=rmclass]
    """
    cat=cat[cat["class"].isin(remove_classes)]
    N = len(cat.index)
    #print('%d events after removing classes' % N)
    return cat, N

def cat_filter_weight(cat, minWeight):
    #if minWeight>0:
    #    cat = cat[cat['weight']>=minWeight]
    cat=cat[cat["weight"].isin(range(minWeight,13))]
    N = len(cat.index)
    #print('%d events after filtering above %d' % (N, minWeight))
    return cat, N

def cat_check_numbers(cat, minthresh = 20):
    df = cat.copy()
    tooSmall = False
    lengths = []
    for subclass in df['class'].unique():
        dfs = df[df['class']==subclass]
        N = len(dfs.index)
        if N<minthresh:
            tooSmall=True
        lengths.append(N)
    return tooSmall, lengths


#######################
# Looping starts here #
#######################
minPerClass = 30
counter = 0
results_list = []
for traceID in traceIDs:
    
    # what traceID are we looking for - read_montserrat needs this - should figure out how to write this into the config
    fptr = open('./AAA-master/MONTSERRAT/current_traceID.txt','w')
    fptr.write(traceID)
    fptr.close()
    
    for minWeight in minWeights:
        for include_classes in classes_to_include:
            # reload cat because we filter it down each time
            #cat = pickle.load(open(output_path_cat,'rb'))
            cat = pd.read_csv(csvfile_internal)
            print(cat['class'].value_counts())
            print(traceID, include_classes, minWeight)
            
            results_dict = {}
            results_dict['traceID'] = traceID
            results_dict['classes'] = ','.join(include_classes)
            results_dict['minWeight'] = minWeight
            results_dict['NtraceID'] = cat_filter_traceID(cat, alltraces, traceID)
            cat, N = cat_filter_classes(cat, include_classes) 
            results_dict['Nclasses'] = N
            cat, N = cat_filter_weight(cat, minWeight)
            results_dict['Nweight'] = N
            tooSmall, lengths = cat_check_numbers(cat, minPerClass)
            results_dict['counts'] = str(lengths)
            
            results_dict['acc_mean'] = None
            results_dict['acc_std'] = None
            
            if N>=minPerClass*len(include_classes) and not tooSmall:
                #try:
                    print(cat.groupby('class').size())
                    analyzer = Analyzer(config, verbatim=verbatim, catalog=cat)
                    allData, allLabels, acc = analyzer.learn(config, returnData=True, verbatim=3) # If you want the data
                    results_dict['acc_mean'] = np.round(np.mean(acc)*100, 1)
                    results_dict['acc_std'] = np.round(np.std(acc)*100, 1)                        
                    cat['predicted_class'] = allLabels
                    cat.to_csv(csvfile_internal.replace('.csv','_predicted_%d.csv' % counter)) 
            results_list.append(results_dict)
            counter += 1
            
resultsDF = pd.DataFrame(results_list)
resultsDF.to_csv('new_results.csv')
print('Done')



/Users/gt/opt/anaconda3/envs/AAA/lib/python3.7/site-packages/obspy/clients/arclink/__init__.py:178: ObsPyDeprecationWarning: The ArcLink protocol will be deprecated in the near future. Please, use the client contacting the routing service provided by EIDA: https://docs.obspy.org/packages/obspy.clients.fdsn.html#basic-routing-clients-usage
  warnings.warn(msg, category=ObsPyDeprecationWarning)


./AAA-master/config/specific/usecase1_continuous_classification/usecase1_EXAMPLE.json
Welcome to this automatic analysis architecture
Copyright: Marielle MALFANTE - GIPSA-Lab
Univ. Grenoble Alpes, CNRS, Grenoble INP, GIPSA-lab, 38000 Grenoble, France

 *** PROJECT CONFIGURATION 10 ***  
Configuration object from <path> ./AAA-master/config/general/newsettings_10.json, <configuration_number> 10,
 <general> {'project_root': './AAA-master/', 'analysis_type': 'continuous', 'path_to_specific_settings_file': 'config/specific/usecase1_continuous_classification/usecase1_EXAMPLE.json', 'path_to_res': 'res/', 'path_to_visuals': 'fig/', 'path_to_res_to_review': 'res_to_review/'},
 <application> {'name': 'montserrat'},
 <preprocessing> {'energy_norm': True},
 <learning> {'algo': RandomForestClassifier(criterion='entropy'), 'cv': StratifiedShuffleSplit(n_splits=30, random_state=None, test_size=0.5,
            train_size=0.5), 'path_to_catalogue': 'catalog/30_MVO_labelled_events_filtered.pd'},
 <fea

Got 15 traces from miniseed_c/MVOE_/2000/01/2000-01-16-1422-01S.MVO___019.mseed
1 traces match MV.MBWH..SHZ
1 Trace(s) in Stream:
MV.MBWH..SHZ | 2000-01-16T14:22:01.040000Z - 2000-01-16T14:23:25.026667Z | 75.0 Hz, 6300 samples
75.0 6300
Got 14 traces from miniseed_c/MVOE_/2000/01/2000-01-18-1413-27S.MVO___019.mseed
1 traces match MV.MBWH..SHZ
1 Trace(s) in Stream:
MV.MBWH..SHZ | 2000-01-18T14:13:27.040000Z - 2000-01-18T14:14:02.026667Z | 75.0 Hz, 2625 samples
75.0 2625
Got 15 traces from miniseed_c/MVOE_/2000/01/2000-01-19-0015-00S.MVO___019.mseed
1 traces match MV.MBWH..SHZ
1 Trace(s) in Stream:
MV.MBWH..SHZ | 2000-01-19T00:15:00.040000Z - 2000-01-19T00:15:54.026667Z | 75.0 Hz, 4050 samples
75.0 4050
Got 15 traces from miniseed_c/MVOE_/2000/01/2000-01-19-0030-40S.MVO___019.mseed
1 traces match MV.MBWH..SHZ
1 Trace(s) in Stream:
MV.MBWH..SHZ | 2000-01-19T00:30:40.040000Z - 2000-01-19T00:31:17.026667Z | 75.0 Hz, 2775 samples
75.0 2775
Got 15 traces from miniseed_c/MVOE_/2000/01/2000-01-

Got 14 traces from miniseed_c/MVOE_/2000/02/2000-02-02-1253-52S.MVO___019.mseed
1 traces match MV.MBWH..SHZ
1 Trace(s) in Stream:
MV.MBWH..SHZ | 2000-02-02T12:53:52.040000Z - 2000-02-02T12:54:40.026667Z | 75.0 Hz, 3600 samples
75.0 3600
Got 14 traces from miniseed_c/MVOE_/2000/02/2000-02-02-1320-19S.MVO___019.mseed
1 traces match MV.MBWH..SHZ
1 Trace(s) in Stream:
MV.MBWH..SHZ | 2000-02-02T13:20:19.040000Z - 2000-02-02T13:20:54.026667Z | 75.0 Hz, 2625 samples
75.0 2625
Got 14 traces from miniseed_c/MVOE_/2000/02/2000-02-02-1338-49S.MVO___019.mseed
1 traces match MV.MBWH..SHZ
1 Trace(s) in Stream:
MV.MBWH..SHZ | 2000-02-02T13:38:49.040000Z - 2000-02-02T13:39:25.026667Z | 75.0 Hz, 2700 samples
75.0 2700
Got 15 traces from miniseed_c/MVOE_/2000/02/2000-02-03-0552-45S.MVO___019.mseed
1 traces match MV.MBWH..SHZ
1 Trace(s) in Stream:
MV.MBWH..SHZ | 2000-02-03T05:52:45.040000Z - 2000-02-03T05:53:21.026667Z | 75.0 Hz, 2700 samples
75.0 2700
Got 15 traces from miniseed_c/MVOE_/2000/02/2000-02-

Got 14 traces from miniseed_c/MVOE_/2000/06/2000-06-09-1831-41S.MVO___019.mseed
1 traces match MV.MBWH..SHZ
1 Trace(s) in Stream:
MV.MBWH..SHZ | 2000-06-09T18:31:41.040000Z - 2000-06-09T18:33:15.026667Z | 75.0 Hz, 7050 samples
75.0 7050
Got 14 traces from miniseed_c/MVOE_/2000/06/2000-06-10-0639-18S.MVO___019.mseed
1 traces match MV.MBWH..SHZ
1 Trace(s) in Stream:
MV.MBWH..SHZ | 2000-06-10T06:39:18.040000Z - 2000-06-10T06:40:18.026667Z | 75.0 Hz, 4500 samples
75.0 4500
Got 14 traces from miniseed_c/MVOE_/2000/06/2000-06-10-0904-05S.MVO___019.mseed
1 traces match MV.MBWH..SHZ
1 Trace(s) in Stream:
MV.MBWH..SHZ | 2000-06-10T09:04:05.040000Z - 2000-06-10T09:05:20.026667Z | 75.0 Hz, 5625 samples
75.0 5625
Got 14 traces from miniseed_c/MVOE_/2000/06/2000-06-11-1903-35S.MVO___019.mseed
1 traces match MV.MBWH..SHZ
1 Trace(s) in Stream:
MV.MBWH..SHZ | 2000-06-11T19:03:35.040000Z - 2000-06-11T19:04:32.026667Z | 75.0 Hz, 4275 samples
75.0 4275
Got 10 traces from miniseed_c/MVOE_/2000/06/2000-06-

...

Got 15 traces from miniseed_c/MVOE_/2000/01/2000-01-16-1456-43S.MVO___019.mseed
1 traces match MV.MBWH..SHZ
1 Trace(s) in Stream:
MV.MBWH..SHZ | 2000-01-16T14:56:43.040000Z - 2000-01-16T14:57:37.026667Z | 75.0 Hz, 4050 samples
75.0 4050
Got 15 traces from miniseed_c/MVOE_/2000/01/2000-01-16-1503-28S.MVO___019.mseed
1 traces match MV.MBWH..SHZ
1 Trace(s) in Stream:
MV.MBWH..SHZ | 2000-01-16T15:03:28.040000Z - 2000-01-16T15:04:14.026667Z | 75.0 Hz, 3450 samples
75.0 3450
Got 15 traces from miniseed_c/MVOE_/2000/01/2000-01-16-1510-02S.MVO___019.mseed
1 traces match MV.MBWH..SHZ
1 Trace(s) in Stream:
MV.MBWH..SHZ | 2000-01-16T15:10:02.040000Z - 2000-01-16T15:11:21.026667Z | 75.0 Hz, 5925 samples
75.0 5925
Got 13 traces from miniseed_c/MVOE_/2000/01/2000-01-16-1524-30S.MVO___019.mseed
1 traces match MV.MBWH..SHZ
1 Trace(s) in Stream:
MV.MBWH..SHZ | 2000-01-16T15:24:30.040000Z - 2000-01-16T15:25:00.026667Z | 75.0 Hz, 2250 samples
75.0 2250
Got 15 traces from miniseed_c/MVOE_/2000/01/2000-01-

75.0 3600
Got 15 traces from miniseed_c/MVOE_/2000/02/2000-02-07-0707-02S.MVO___019.mseed
1 traces match MV.MBWH..SHZ
1 Trace(s) in Stream:
MV.MBWH..SHZ | 2000-02-07T07:07:02.040000Z - 2000-02-07T07:07:47.026667Z | 75.0 Hz, 3375 samples
75.0 3375
Got 15 traces from miniseed_c/MVOE_/2000/02/2000-02-07-0750-59S.MVO___019.mseed
1 traces match MV.MBWH..SHZ
1 Trace(s) in Stream:
MV.MBWH..SHZ | 2000-02-07T07:50:59.040000Z - 2000-02-07T07:51:53.026667Z | 75.0 Hz, 4050 samples
75.0 4050
Got 15 traces from miniseed_c/MVOE_/2000/02/2000-02-07-0904-21S.MVO___019.mseed
1 traces match MV.MBWH..SHZ
1 Trace(s) in Stream:
MV.MBWH..SHZ | 2000-02-07T09:04:21.040000Z - 2000-02-07T09:05:06.026667Z | 75.0 Hz, 3375 samples
75.0 3375
Got 15 traces from miniseed_c/MVOE_/2000/02/2000-02-07-0907-05S.MVO___019.mseed
1 traces match MV.MBWH..SHZ
1 Trace(s) in Stream:
MV.MBWH..SHZ | 2000-02-07T09:07:05.040000Z - 2000-02-07T09:07:44.026667Z | 75.0 Hz, 2925 samples
75.0 2925
Got 15 traces from miniseed_c/MVOE_/2000/0

Got 14 traces from miniseed_c/MVOE_/2000/01/2000-01-15-0556-14S.MVO___019.mseed
1 traces match MV.MBWH..SHZ
1 Trace(s) in Stream:
MV.MBWH..SHZ | 2000-01-15T05:56:14.040000Z - 2000-01-15T05:56:44.026667Z | 75.0 Hz, 2250 samples
75.0 2250
Got 15 traces from miniseed_c/MVOE_/2000/01/2000-01-15-1338-03S.MVO___019.mseed
1 traces match MV.MBWH..SHZ
1 Trace(s) in Stream:
MV.MBWH..SHZ | 2000-01-15T13:38:03.040000Z - 2000-01-15T13:38:51.026667Z | 75.0 Hz, 3600 samples
75.0 3600
Got 15 traces from miniseed_c/MVOE_/2000/01/2000-01-15-1354-02S.MVO___019.mseed
1 traces match MV.MBWH..SHZ
1 Trace(s) in Stream:
MV.MBWH..SHZ | 2000-01-15T13:54:02.040000Z - 2000-01-15T13:55:19.026667Z | 75.0 Hz, 5775 samples
75.0 5775
Got 15 traces from miniseed_c/MVOE_/2000/01/2000-01-18-1329-17S.MVO___019.mseed
1 traces match MV.MBWH..SHZ
1 Trace(s) in Stream:
MV.MBWH..SHZ | 2000-01-18T13:29:17.040000Z - 2000-01-18T13:29:48.026667Z | 75.0 Hz, 2325 samples
75.0 2325
Got 15 traces from miniseed_c/MVOE_/2000/01/2000-01-

Got 14 traces from miniseed_c/MVOE_/2000/02/2000-02-02-0134-46S.MVO___019.mseed
1 traces match MV.MBWH..SHZ
1 Trace(s) in Stream:
MV.MBWH..SHZ | 2000-02-02T01:34:46.040000Z - 2000-02-02T01:35:45.026667Z | 75.0 Hz, 4425 samples
75.0 4425
Got 15 traces from miniseed_c/MVOE_/2000/02/2000-02-03-0709-19S.MVO___019.mseed
1 traces match MV.MBWH..SHZ
1 Trace(s) in Stream:
MV.MBWH..SHZ | 2000-02-03T07:09:19.040000Z - 2000-02-03T07:10:36.026667Z | 75.0 Hz, 5775 samples
75.0 5775
Got 14 traces from miniseed_c/MVOE_/2000/02/2000-02-03-2150-21S.MVO___019.mseed
1 traces match MV.MBWH..SHZ
1 Trace(s) in Stream:
MV.MBWH..SHZ | 2000-02-03T21:50:21.040000Z - 2000-02-03T21:51:17.026667Z | 75.0 Hz, 4200 samples
75.0 4200
Got 14 traces from miniseed_c/MVOE_/2000/02/2000-02-05-2047-53S.MVO___019.mseed
1 traces match MV.MBWH..SHZ
1 Trace(s) in Stream:
MV.MBWH..SHZ | 2000-02-05T20:47:53.040000Z - 2000-02-05T20:48:22.026667Z | 75.0 Hz, 2175 samples
75.0 2175
Got 13 traces from miniseed_c/MVOE_/2000/02/2000-02-

Got 14 traces from miniseed_c/MVOE_/2000/03/2000-03-06-0840-42S.MVO___019.mseed
1 traces match MV.MBWH..SHZ
1 Trace(s) in Stream:
MV.MBWH..SHZ | 2000-03-06T08:40:42.040000Z - 2000-03-06T08:41:26.026667Z | 75.0 Hz, 3300 samples
75.0 3300
Got 15 traces from miniseed_c/MVOE_/2000/03/2000-03-06-1024-55S.MVO___019.mseed
1 traces match MV.MBWH..SHZ
1 Trace(s) in Stream:
MV.MBWH..SHZ | 2000-03-06T10:24:55.040000Z - 2000-03-06T10:25:43.026667Z | 75.0 Hz, 3600 samples
75.0 3600
Got 15 traces from miniseed_c/MVOE_/2000/03/2000-03-06-1036-28S.MVO___019.mseed
1 traces match MV.MBWH..SHZ
1 Trace(s) in Stream:
MV.MBWH..SHZ | 2000-03-06T10:36:28.040000Z - 2000-03-06T10:37:30.026667Z | 75.0 Hz, 4650 samples
75.0 4650
Got 15 traces from miniseed_c/MVOE_/2000/03/2000-03-06-1615-43S.MVO___019.mseed
1 traces match MV.MBWH..SHZ
1 Trace(s) in Stream:
MV.MBWH..SHZ | 2000-03-06T16:15:43.040000Z - 2000-03-06T16:17:28.026667Z | 75.0 Hz, 7875 samples
75.0 7875
Got 15 traces from miniseed_c/MVOE_/2000/03/2000-03-